In [6]:
import pandas as pd
from pyproj import Transformer
import geopandas as gpd
import folium
from geopy.geocoders import Nominatim
from geopy.distance import geodesic

#Récuperation et traitement des données
dataDREAL = pd.read_csv('C:/Users/aguit/OneDrive/Documents/VSCODE/Unilasalle/Mineur Traitement de données/Data/Exploit_DREAL_CSV.csv', encoding='ISO-8859-1',  sep=';')
data_loc = dataDREAL.iloc[:, [2, 3, 4, 5,8,9,24,30,31]]
data_loc = data_loc.dropna(subset=['EXPLOITANT'], inplace=False)
data_loc = data_loc.dropna(subset=['NAT_EXPLOI'], inplace=False)
data_loc['x'] = pd.to_numeric(data_loc['x'].str.replace(',', '.'))
data_loc['y'] = pd.to_numeric(data_loc['y'].str.replace(',', '.'))
test1 = data_loc
test1['AP_DAT_DEB'] = pd.to_datetime(test1['AP_DAT_DEB'], errors='coerce')
test1 = test1[test1['AP_DAT_DEB'].dt.year > 2010]
test1 = test1.loc[test1['STATUT'] == 'Exploitation autorisée']
f_gra = test1[test1['NAT_EXPLOI'].str.contains("à")]
f_mat = test1[test1['NAT_EXPLOI'].str.contains("Matériaux")]
f_ind = test1[test1['NAT_EXPLOI'].str.contains("industrie")]
f_orn = test1[test1['NAT_EXPLOI'].str.contains("ornementales")]

# Choisir une ville
ville_choisie = "Paris"
# Choisir le rayon
rayon_km = 10000
materiaux = "Granulats (0,08mm à 80mm)"
# Création de la carte
geolocator = Nominatim(user_agent="my_geocoder")
location = geolocator.geocode(ville_choisie)
ville_coords = (location.latitude, location.longitude)
carte = folium.Map([46.603354, 1.888334], zoom_start=5.5)
transformer = Transformer.from_crs("epsg:2154", "epsg:4326")

# Définition des icones
icon_mapping = {'Granulats (0,08mm à 80mm)': 'C:/Users/aguit/OneDrive/Documents/VSCODE/Unilasalle/Mineur Traitement de données/Data/Icone1.png',
                "Roches et minéraux pour l'industrie": 'C:/Users/aguit/OneDrive/Documents/VSCODE/Unilasalle/Mineur Traitement de données/Data/Icone2.png',
                "Roches ornementales et de construction" : 'C:/Users/aguit/OneDrive/Documents/VSCODE/Unilasalle/Mineur Traitement de données/Data/Icone3.png',
                "Matériaux > 80mm" : 'C:/Users/aguit/OneDrive/Documents/VSCODE/Unilasalle/Mineur Traitement de données/Data/Icone4.png',
                "Granulats (0,08mm à 80mm), Roches ornementales et de construction" : 'C:/Users/aguit/OneDrive/Documents/VSCODE/Unilasalle/Mineur Traitement de données/Data/Icone5.png',
                "Granulats (0,08mm à 80mm), Matériaux > 80mm" : 'C:/Users/aguit/OneDrive/Documents/VSCODE/Unilasalle/Mineur Traitement de données/Data/Icone6.png',
                "Roches et minéraux pour l'industrie, Roches ornementales et de construction" : 'C:/Users/aguit/OneDrive/Documents/VSCODE/Unilasalle/Mineur Traitement de données/Data/Icone7.png',
                "Granulats (0,08mm à 80mm), Roches et minéraux pour l'industrie" : "C:/Users/aguit/OneDrive/Documents/VSCODE/Unilasalle/Mineur Traitement de données/Data/Icone8.png",
                "Granulats (0,08mm à 80mm), Roches et minéraux pour l'industrie, Roches ornementales et de construction" :  'C:/Users/aguit/OneDrive/Documents/VSCODE/Unilasalle/Mineur Traitement de données/Data/Icone9.png',
                "Granulats (0,08mm à 80mm), Matériaux > 80mm, Roches ornementales et de construction" : 'C:/Users/aguit/OneDrive/Documents/VSCODE/Unilasalle/Mineur Traitement de données/Data/Icone13.png',
                "Matériaux > 80mm, Roches et minéraux pour l'industrie" : "C:/Users/aguit/OneDrive/Documents/VSCODE/Unilasalle/Mineur Traitement de données/Data/Icone11.png",
                "Matériaux > 80mm, Roches ornementales et de construction" :"C:/Users/aguit/OneDrive/Documents/VSCODE/Unilasalle/Mineur Traitement de données/Data/Icone12.png",
                "Granulats (0,08mm à 80mm), Matériaux > 80mm, Roches et minéraux pour l'industrie" : "C:/Users/aguit/OneDrive/Documents/VSCODE/Unilasalle/Mineur Traitement de données/Data/Icone10.png"}

if  materiaux == 'Tous' :
    latlong_origin = list(zip(*transformer.transform(test1["x"], test1["y"])))
    # Filtrer les points à l'intérieur du rayon autour de la ville choisie
    for origin, exploitant, nat_exploi in zip(latlong_origin, test1['EXPLOITANT'], test1['NAT_EXPLOI']):
        distance = geodesic(ville_coords, origin).kilometers
        if distance <= rayon_km:            
            icon_path = icon_mapping.get(nat_exploi, 'C:/Users/aguit/OneDrive/Documents/VSCODE/Unilasalle/Mineur Traitement de données/Data/Carriere.png')
            pop = f"Exploitant: {exploitant}<br/>Distance: {distance:.2f} km<br/>Type de materiaux: {nat_exploi}"
            icon = folium.CustomIcon(icon_path, icon_size=(20, 20), shadow_size=(0, 0))
            folium.Marker(origin, popup=pop, icon=icon).add_to(carte)
    carte.save("C:/Users/aguit/OneDrive/Documents/VSCODE/Unilasalle/Mineur Traitement de données/carte_carrieres.html")  # Sauvegarder la carte au format HTML


elif  materiaux == 'Granulats (0,08mm à 80mm)' :
    latlong_origin = list(zip(*transformer.transform(f_gra["x"], f_gra["y"])))
    # Filtrer les points à l'intérieur du rayon autour de la ville choisie
    for origin, exploitant, nat_exploi in zip(latlong_origin, f_gra['EXPLOITANT'], f_gra['NAT_EXPLOI']):
        distance = geodesic(ville_coords, origin).kilometers
        if distance <= rayon_km:
            icon_path = icon_mapping.get(nat_exploi, 'C:/Users/aguit/OneDrive/Documents/VSCODE/Unilasalle/Mineur Traitement de données/Data/Carriere.png')
            pop = f"Exploitant: {exploitant}<br/>Distance: {distance:.2f} km<br/>Type de materiaux: {nat_exploi}"
            icon = folium.CustomIcon(icon_path, icon_size=(20, 20), shadow_size=(0, 0))
            folium.Marker(origin, popup=pop, icon=icon).add_to(carte)
            #folium.Circle(location=ville_coords, radius=rayon_km*1000, color='blue', fill=False).add_to(carte)
    # Afficher la carte
    carte.save("C:/Users/aguit/OneDrive/Documents/VSCODE/Unilasalle/Mineur Traitement de données/carte_granulats_inf_80mm.html")  # Sauvegarder la carte au format HTML

elif materiaux == "Roches et minéraux pour l'industrie" :
    latlong_origin = list(zip(*transformer.transform(f_ind["x"], f_ind["y"])))
    # Filtrer les points à l'intérieur du rayon autour de la ville choisie
    for origin, exploitant, nat_exploi in zip(latlong_origin, f_ind['EXPLOITANT'], f_ind['NAT_EXPLOI']):
        distance = geodesic(ville_coords, origin).kilometers
        if distance <= rayon_km:
            icon_path = icon_mapping.get(nat_exploi, 'C:/Users/aguit/OneDrive/Documents/VSCODE/Unilasalle/Mineur Traitement de données/Data/Carriere.png')
            pop = f"Exploitant: {exploitant}<br/>Distance: {distance:.2f} km<br/>Type de materiaux: {nat_exploi}"
            icon = folium.CustomIcon(icon_path, icon_size=(20, 20), shadow_size=(0, 0))
            folium.Marker(origin, popup=pop, icon=icon).add_to(carte)
            #folium.Circle(location=ville_coords, radius=rayon_km*1000, color='blue', fill=False).add_to(carte)
    # Afficher la carte
    carte.save("C:/Users/aguit/OneDrive/Documents/VSCODE/Unilasalle/Mineur Traitement de données/carte_roches_et_mineraux_industrie.html")  # Sauvegarder la carte au format HTML

elif materiaux == "Matériaux > 80mm" :
    latlong_origin = list(zip(*transformer.transform(f_mat["x"], f_mat["y"])))
    # Filtrer les points à l'intérieur du rayon autour de la ville choisie
    for origin, exploitant, nat_exploi in zip(latlong_origin, f_mat['EXPLOITANT'], f_mat['NAT_EXPLOI']):
        distance = geodesic(ville_coords, origin).kilometers
        if distance <= rayon_km:
            icon_path = icon_mapping.get(nat_exploi, 'C:/Users/aguit/OneDrive/Documents/VSCODE/Unilasalle/Mineur Traitement de données/Data/Carriere.png')
            pop = f"Exploitant: {exploitant}<br/>Distance: {distance:.2f} km<br/>Type de materiaux: {nat_exploi}"
            icon = folium.CustomIcon(icon_path, icon_size=(20, 20), shadow_size=(0, 0))
            folium.Marker(origin, popup=pop, icon=icon).add_to(carte)
            #folium.Circle(location=ville_coords, radius=rayon_km*1000, color='blue', fill=False).add_to(carte)
    # Afficher la carte
    carte.save("C:/Users/aguit/OneDrive/Documents/VSCODE/Unilasalle/Mineur Traitement de données/carte_materiaux_sup_80mm.html")  # Sauvegarder la carte au format HTML
elif materiaux =="Roches ornementales et de construction"  :
    latlong_origin = list(zip(*transformer.transform(f_orn["x"], f_orn["y"])))
    # Filtrer les points à l'intérieur du rayon autour de la ville choisie
    for origin, exploitant, nat_exploi in zip(latlong_origin, f_orn['EXPLOITANT'], f_orn['NAT_EXPLOI']):
        distance = geodesic(ville_coords, origin).kilometers
        if distance <= rayon_km:
            icon_path = icon_mapping.get(nat_exploi, 'C:/Users/aguit/OneDrive/Documents/VSCODE/Unilasalle/Mineur Traitement de données/Data/Carriere.png')
            pop = f"Exploitant: {exploitant}<br/>Distance: {distance:.2f} km<br/>Type de materiaux: {nat_exploi}"
            icon = folium.CustomIcon(icon_path, icon_size=(20, 20), shadow_size=(0, 0))
            folium.Marker(origin, popup=pop, icon=icon).add_to(carte)
            #folium.Circle(location=ville_coords, radius=rayon_km*1000, color='blue', fill=False).add_to(carte)
    # Afficher la carte
    carte.save("C:/Users/aguit/OneDrive/Documents/VSCODE/Unilasalle/Mineur Traitement de données/carte_roches_ornementales_et_construction.html")  # Sauvegarder la carte au format HTML

C:\Users\aguit\AppData\Local\Temp\ipykernel_143508\1458235734.py:16: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  test1['AP_DAT_DEB'] = pd.to_datetime(test1['AP_DAT_DEB'], errors='coerce')
